This notebook is not needed anymore. It was used to estimate additional scaling parameters for INH EXC and GLIAL cells to match cell densities to Sebastien's validation script. 

The resulting parameters are then applied in the 3rd part of the density1.5_multi_scaling step. 

In [1]:
#Newer pickle versions are not compatible
!which python
import pandas as pd
print(pd.__version__)
import sys
print(sys.version)
import inspect
print(inspect.getfile(pd))

/gpfs/bbp.cscs.ch/ssd/apps/bsd/2024-02-01/stage_externals/install_gcc-12.3.0-skylake/python-3.11.6-bj4i6m/bin/python
2.2.1
3.12.2 | packaged by Anaconda, Inc. | (main, Feb 27 2024, 17:35:02) [GCC 11.2.0]
/gpfs/bbp.cscs.ch/data/project/proj84/csaba/git/cellden/lib/python3.12/site-packages/pandas/__init__.py


In [2]:
import numpy as np
import pandas as pd
import pickle
import nrrd
import multiprocessing as mp
import re, os, copy
from voxcell import RegionMap
import ast

import sys
sys.path.append('/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/notebooks/scripts/')

from helper_functions import get_all_filenames, get_csv_filenames, extract_prefix_from_filenames, read_and_concat_csv_files_new, combine_rows_and_calculate_average, create_combined_dataframe

In [3]:
download_base = '/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/'
root_folder = f"{download_base}results/density_calculations/"

# Load estimated non-scaled densities

In [4]:
%%time 

# Get all regional density data
folder_path = f"{root_folder}csv/"
filenames = get_all_filenames(folder_path)
csv_filenames = get_csv_filenames(folder_path)
prefixes = extract_prefix_from_filenames(csv_filenames)
unique_prefixes = sorted(list(set(prefixes)))

#Create a dict of df, each containing a cell type's occurence in all regions and its densities in all regions
result_dataframes = read_and_concat_csv_files_new(csv_filenames, unique_prefixes, folder_path)
combined_result_dataframes = combine_rows_and_calculate_average(result_dataframes)
shuffled_combined_dataframes = create_combined_dataframe(combined_result_dataframes)

file = '/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/metadata/parcellation_to_parcellation_term_membership_extend.csv'
parcellation_annotation = pd.read_csv(file)
print("number of memberships at all levels:",len(parcellation_annotation))
parcellation_annotation = parcellation_annotation[parcellation_annotation['parcellation_term_set_name'] == 'substructure'] 
print("number of memberships at substructure level:",len(parcellation_annotation))
volumes = parcellation_annotation[['parcellation_label', 'parcellation_index', 'parcellation_term_acronym', 'parcellation_term_set_name', 'parcellation_term_name', 'voxel_count', 'volume_mm3', 'label_numbers', 'cluster_as_filename']]
substructure_vol = volumes[volumes['parcellation_term_set_name'] == 'substructure']



number of memberships at all levels: 3489
number of memberships at substructure level: 737
CPU times: user 37.1 s, sys: 142 ms, total: 37.3 s
Wall time: 37.2 s


In [5]:
len(shuffled_combined_dataframes)

5274

In [6]:
#Read CCFv3 annotation volumes (choose 1) for calculating total cell counts
# data_folder = "/gpfs/bbp.cscs.ch/project/proj84/piluso/share/general/warped_augmented_CCFv3/"
# CCFv3, _ = nrrd.read(f'{data_folder}annotation_25_2022_CCFv3.nrrd')
# data_folder = "/gpfs/bbp.cscs.ch/data/project/proj62/csaba/atlas/bbp_prod_files/2022/"
# CCFv3, _ = nrrd.read(f'{data_folder}annotation_25.nrrd')
data_folder = "/gpfs/bbp.cscs.ch/project/proj84/piluso/share/general/warped_augmented_CCFv3/"
CCFv3, _ = nrrd.read(f'{data_folder}annotation_25_2022_CCFv3a.nrrd')

# Create float copy of the annotation voluem, update values in CCFv3 based on conditions
#CCFv3_copy = np.copy(CCFv3).astype('float64')

#hierarchy_json = '/gpfs/bbp.cscs.ch/home/veraszto/bbp_prod_files/1.json'
hierarchy_json = '/gpfs/bbp.cscs.ch/data/project/proj84/atlas_pipeline_runs/2024-05-15T22:44:26+02:00/hierarchy_ccfv3_l23split_barrelsplit.json'
region_map = RegionMap.load_json(hierarchy_json)


# Local (area by area) scaling when necessary
Scaling parameters are provided by Blue Brain in the next cell

In [7]:
# Create a dictionary from the expected data:
data = {
    "id": [391, 399, 407, 415, 431, 438, 446, 454, 471, 486, 495, 504, 718, 733, 1020],
    "acronym": [
        "CA1slm", "CA1so", "CA1sp", "CA1sr", "CA2slm", "CA2so", 
        "CA2sp", "CA2sr", "CA3slm", "CA3so", "CA3sp", "CA3sr", "VPL", "VPM", "PO"
    ],
    "Region": [
        "Field CA1, stratum lacunosum-moleculare", "Field CA1, stratum oriens",
        "Field CA1, stratum pyramidale", "Field CA1, stratum radiatum",
        "Field CA2, stratum lacunosum-moleculare", "Field CA2, stratum oriens",
        "Field CA2, stratum pyramidale", "Field CA2, stratum radiatum",
        "Field CA3, stratum lacunosum-moleculare", "Field CA3, stratum oriens",
        "Field CA3, stratum pyramidale", "Field CA3, stratum radiatum",
        "Ventral posterolateral nucleus of the thalamus", "Ventral posteromedial nucleus of the thalamus",
        "Posterior complex of the thalamus"
    ],
    "Expected Inhibitory Dorsal": [
        8420, 6480, 13520, 6480, 8420, 6480, 13520, 3240, 5780, 3170, 7560, 7890, "", "", ""
    ],
    "Expected Inhibitory Ventral": [
        8820, 10500, 10130, 10500, 8820, 10500, 10130, 4260, 7200, 5460, 10540, 6590, "", "", ""
    ],
    "Expected Neurons Dorsal": [
        "", "", 447500, "", "", "", 447500, "", "", "", "", "", "", "", ""
    ],
    "Expected Neurons Ventral": [
        "", "", 180500, "", "", "", 180500, "", "", "", "", "", "", "", ""
    ],
    "exc_mm3": [
        "", "", "", "", "", "", "", "", "", "", "", "", 55340.64, 67940.58, ""
    ],
    "inh_mm3": [
        8620, 8490, 11825, 8490, 8620, 8490, 11825, 3750, 6490, 4315, 9050, 7240, 2126.28, 2978.61, 161.08663
    ],
    "neurons_mm3": [
        "", "", 314000, "", "", "", 314000, "", "", "", 172400, "", 57466.92, 70919.19, ""
    ]
}

# Create a DataFrame from the dictionary
df_e = pd.DataFrame(data)


# New data to be added
new_data = {
    "id": [212, 220, 228, 236, 244, 477],
    "acronym": [
        "MOBglomerularlayer", "MOBgr", "MOBipl", "MOBmi", "MOBopl", "STR"
    ],
    "Region": [
        "Main olfactory bulb, glomerular layer", "Main olfactory bulb, granule layer",
        "Main olfactory bulb, inner plexiform layer", "Main olfactory bulb, mitral layer",
        "Main olfactory bulb, outer plexiform layer", "Striatum"
    ],
    "Expected Inhibitory Dorsal": ["", "", "", "", "", ""],
    "Expected Inhibitory Ventral": ["", "", "", "", "", ""],
    "Expected Neurons Dorsal": ["", "", "", "", "", ""],
    "Expected Neurons Ventral": ["", "", "", "", "", ""],
    "exc_mm3": ["", "", "", "", "", ""],
    "inh_mm3": ["", "", "", "", "", ""],
    "neurons_mm3": [
        630000, 710000, 150000, 350000, 80000, 78560
    ]
}
df_new = pd.DataFrame(new_data)

# Concatenate the new data with the existing data
df_expected = pd.concat([df_e, df_new], ignore_index=True)
df_expected.to_csv('/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/expected_data.csv', index=False)

#Add children to the regions which need to change to include non-leaf-region changes:
file = '/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/df_hierarchy_ccfv3_l23split_barrelsplit.csv'
hierarchy = pd.read_csv(file, index_col=0)

# Merge df_expected with hierarchy on 'id' to get the 'children' column
df_expected = df_expected.merge(hierarchy[['id', 'children', 'acr_list']], on='id', how='left')

import ast
df_expected.loc[: , 'children'] = df_expected.loc[: , 'children'].apply(ast.literal_eval)

# Add new columns to df_expected
df_expected['exc_mm3_ratio'] = np.nan
df_expected['inh_mm3_ratio'] = np.nan
df_expected['neurons_mm3_ratio'] = np.nan

## Total nrrd creation
We can separate cell types into larger (hierarchical groups) and convert densities to total cell counts. We can save the results as nrrd files. 
Step 1 is to create the hierarchical cell type groups. 

In [8]:
%%time 

meta_path = "/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/metadata/WMB-10X/20231215/views/cell_metadata_with_cluster_annotation.csv"
columns_to_read = ['class', 'subclass', 'cluster']
#metadata = pd.read_csv(meta_path, dtype={'cell_label':str}, low_memory=False)
metadata = pd.read_csv(meta_path, usecols=columns_to_read, )

n_classes = ['01 IT-ET Glut', '02 NP-CT-L6b Glut', '03 OB-CR Glut',
       '04 DG-IMN Glut', '05 OB-IMN GABA', '06 CTX-CGE GABA',
       '07 CTX-MGE GABA', '08 CNU-MGE GABA', '09 CNU-LGE GABA',
       '10 LSX GABA', '11 CNU-HYa GABA', '12 HY GABA', '13 CNU-HYa Glut',
       '14 HY Glut', '15 HY Gnrh1 Glut', '16 HY MM Glut', '17 MH-LH Glut',
       '18 TH Glut', '19 MB Glut', '20 MB GABA', '21 MB Dopa',
       '22 MB-HB Sero', '23 P Glut', '24 MY Glut', '25 Pineal Glut',
       '26 P GABA', '27 MY GABA', '28 CB GABA', '29 CB Glut',]

nn_classes = ['30 Astro-Epen', '31 OPC-Oligo', '32 OEC', '33 Vascular',
       '34 Immune']

exc = ['01 IT-ET Glut', '02 NP-CT-L6b Glut', '03 OB-CR Glut',
      '04 DG-IMN Glut', '13 CNU-HYa Glut', '14 HY Glut', '15 HY Gnrh1 Glut', '16 HY MM Glut', '17 MH-LH Glut',
      '18 TH Glut', '19 MB Glut', '23 P Glut', '24 MY Glut', '25 Pineal Glut', '29 CB Glut',]
inh = ['05 OB-IMN GABA', '06 CTX-CGE GABA', '07 CTX-MGE GABA', '08 CNU-MGE GABA', '09 CNU-LGE GABA',
      '10 LSX GABA', '11 CNU-HYa GABA', '12 HY GABA', '20 MB GABA', '26 P GABA', '27 MY GABA', '28 CB GABA', ]
other = ['21 MB Dopa', '22 MB-HB Sero', ]
exci_inhib_sum = exc + inh

astrotypes = ['5206 Bergmann NN_1', '5207 Astro-CB NN_1', '5208 Astro-NT NN_1',
       '5209 Astro-NT NN_1', '5210 Astro-NT NN_1', '5211 Astro-NT NN_1',
       '5212 Astro-NT NN_1', '5213 Astro-NT NN_1', '5214 Astro-NT NN_2',
       '5215 Astro-NT NN_2', '5216 Astro-NT NN_2', '5217 Astro-NT NN_2',
       '5218 Astro-TE NN_1', '5219 Astro-TE NN_1', '5220 Astro-TE NN_1',
       '5221 Astro-TE NN_1', '5222 Astro-TE NN_2', '5223 Astro-TE NN_2',
       '5224 Astro-TE NN_3', '5225 Astro-TE NN_3', '5226 Astro-TE NN_3',
       '5227 Astro-TE NN_3', '5228 Astro-TE NN_4', '5229 Astro-TE NN_5',
       '5230 Astro-TE NN_5', '5231 Astro-OLF NN_1', '5232 Astro-OLF NN_1',
       '5233 Astro-OLF NN_2', '5234 Astro-OLF NN_2',
       '5235 Astro-OLF NN_3', '5236 Astro-OLF NN_3',]

microglia = ['5312 Microglia NN_1']

oligos = [ '5266 OPC NN_1', '5267 OPC NN_1',
       '5268 OPC NN_1', '5269 OPC NN_1', '5270 OPC NN_1', '5271 OPC NN_2',
       '5272 COP NN_1', '5273 COP NN_1', '5274 COP NN_1', '5275 COP NN_1',
       '5276 COP NN_1', '5277 COP NN_1', '5278 NFOL NN_2',
       '5279 NFOL NN_2', '5280 NFOL NN_2', '5281 NFOL NN_2',
       '5282 MFOL NN_3', '5283 MFOL NN_3', '5284 MOL NN_4',
       '5285 MOL NN_4', '5286 MOL NN_4', '5287 MOL NN_4', '5288 MOL NN_4',]

glia = astrotypes + microglia + oligos

neurontypes = np.unique(metadata[metadata['class'].isin(n_classes)]['cluster'].values)
nonneurontypes = np.unique(metadata[metadata['class'].isin(nn_classes)]['cluster'].values)
exctypes = np.unique(metadata[metadata['class'].isin(exc)]['cluster'].values)
inhtypes = np.unique(metadata[metadata['class'].isin(inh)]['cluster'].values)
othertypes = np.unique(metadata[metadata['class'].isin(other)]['cluster'].values)
exci_inhib_sum = np.unique(metadata[metadata['class'].isin(exci_inhib_sum)]['cluster'].values)
celltypes = np.unique(metadata['cluster'].values)

CPU times: user 16.3 s, sys: 379 ms, total: 16.7 s
Wall time: 16.7 s


In [9]:
#Helper function to create total cell count values for a 3D brain

# import re
# import pandas as pd
# import numpy as np

def nrrd_for_validation(df, parcellation_annotation, CCFv3):
    all_ids_for_df = []
    df_comb = pd.DataFrame()

    for regionname in df.index.values[0:]:
        density = df.loc[regionname, 'density_mm3']
        #annotation_id_info = substructures[substructures['cluster_as_filename'] == regionname]
        annotation_id_info = parcellation_annotation[parcellation_annotation['cluster_as_filename'] == regionname]

        Annotation2020ids = [int(re.search(r'\d+$', s).group()) for s in annotation_id_info['parcellation_label'].values]
        df_sub = pd.DataFrame({'density': density}, index=Annotation2020ids)
        df_comb = pd.concat([df_comb, df_sub])
        all_ids_for_df.append(Annotation2020ids)

    all_ids_for_df = [value for sublist in all_ids_for_df for value in sublist]
    all_ids_for_df.append(0)
    #Place to put extra regions not part of Allen's Parcellation annotation

    outside = 0
    outsideid = [0]
    df_sub = pd.DataFrame({'density': outside}, index=outsideid)
    df_comb = pd.concat([df_comb, df_sub])

    CCFv3_copy = CCFv3.copy()

    # Expression is 0 in those regions where we don't have any info:
    CCFv3_copy[~np.isin(CCFv3_copy, all_ids_for_df)] = 0.0 

    # Expression is non-zero in these leaf region(s)
    for index, row in df_comb.iterrows():
        density_value = row['density']
        region_id = index
        CCFv3_copy[np.isin(CCFv3, region_id)] = density_value

    #Create outside of the brain as 0
    CCFv3_copy[np.isin(CCFv3, int(0))] = 0

    return CCFv3_copy

In [10]:
%%time

#from multiprocessing import Pool

def process_type(types, file_name):
    # Filter DataFrames based on types
    filtered_dataframes = {key: value for key, value in shuffled_combined_dataframes.items() if key in types}
    
    # Combine filtered DataFrames
    combined_df = pd.concat(filtered_dataframes.values())
    
    # Sum the combined DataFrame by index
    summed_df = combined_df.groupby(combined_df.index).sum()
    
    # Validate result
    result = nrrd_for_validation(summed_df, parcellation_annotation, CCFv3)
    
    # Clean up
    del combined_df, summed_df, filtered_dataframes

    return (result, file_name)

def main():
    # Define the parameters for each process
    tasks = [
        (neurontypes, "total_neuron_densities"),
        (exctypes, "total_excitatory_densities"),
        (inhtypes, "total_inhibitory_densities"),
        (astrotypes, "total_astrotypes_densities"),
        (microglia, "total_microglia_densities"),
        (oligos, "total_oligocyte_densities"),
        (glia, "total_glia_densities"),
        (exci_inhib_sum, "total_excinh_densities"),
        (celltypes, "total_celltypes_densities"),
    ]
    
    # Create a multiprocessing Pool
    with mp.Pool(processes=mp.cpu_count()) as pool:
        results = pool.starmap(process_type, tasks)
    
    # Sequentially write the .nrrd files to avoid concurrent writes
    for result, file_name in results:
        nrrd.write(f"{root_folder}{file_name}.nrrd", result)
        print(f"{root_folder}{file_name}.nrrd")
        
if __name__ == "__main__":
    main()

/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations_3/total_neuron_densities.nrrd
/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations_3/total_excitatory_densities.nrrd
/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations_3/total_inhibitory_densities.nrrd
/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations_3/total_astrotypes_densities.nrrd
/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations_3/total_microglia_densities.nrrd
/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations_3/total_oligocyte_densities.nrrd
/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations_3/total_glia_densities.nrrd
/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/den

In [10]:
#Add scaling parameter to df_expected too by loading scaled total density files and search for corresponding values (computed earlier)

# data_folder = "/gpfs/bbp.cscs.ch/project/proj84/piluso/share/general/warped_augmented_CCFv3/"
# #CCFv3a, _ = nrrd.read(f'{data_folder}annotation_25_2022_CCFv3a.nrrd')
# CCFv3a, _ = nrrd.read("/gpfs/bbp.cscs.ch/data/project/proj84/atlas_pipeline_runs/2024-05-15T22:44:26+02:00/annotation_ccfv3_l23split_barrelsplit_validated.nrrd")


#NEURON
filename = "total_neuron_densities.nrrd"
full_path = os.path.join(root_folder, filename)
if os.path.isfile(full_path):
    neuron_total, _ = nrrd.read(full_path)

#EXC
filename = "total_excitatory_densities.nrrd"
full_path = os.path.join(root_folder, filename)
if os.path.isfile(full_path):
    exc_total, _ = nrrd.read(full_path)
#INH        
filename = "total_inhibitory_densities.nrrd"
full_path = os.path.join(root_folder, filename)
if os.path.isfile(full_path):
    inh_total, _ = nrrd.read(full_path)


In [12]:
neuron_total.shape, exc_total.shape, inh_total.shape

((566, 320, 456), (566, 320, 456), (566, 320, 456))

In [13]:
%%time 

''' Calculating scaling parameters: We divide by "mean expected / transcriptomic mean densities". '''

# Arrays to process
arrays = {
    'neurons_mm3': neuron_total,
    'inh_mm3': inh_total,
    'exc_mm3': exc_total,
}

# Worker function to process each row
def process_row(row):
    id_ = row['id'] 
    acr = row['acronym']
    name = row['Region']
    ids = row['children']
    ids.append(id_) #Make sure you add the original id_ to the children
    
    
    # Create a mask for the current id list
    mask = np.isin(CCFv3, [ids])
    
    # Calculate mean/median values (can place NaN or None in the regions not present in CCfV3)
    # mean_values = {name: arr[mask].mean() if arr[mask].size > 
    #                0 else np.nan for name, arr in arrays.items()}
    mean_values = {name: arr[mask].mean() if arr[mask].size > 
                   0 else None for name, arr in arrays.items()}
    # mean_values = {name: np.median(arr[mask]) if arr[mask].size > 
    #                0 else None for name, arr in arrays.items()}

    # Update row with ratio values
    if mean_values['exc_mm3'] is not np.nan and isinstance(row['exc_mm3'], (int, float)):
        row['exc_mm3_ratio'] = row['exc_mm3'] / mean_values['exc_mm3']
    if mean_values['inh_mm3'] is not np.nan and isinstance(row['inh_mm3'], (int, float)):
        row['inh_mm3_ratio'] = row['inh_mm3'] / mean_values['inh_mm3']
    if mean_values['neurons_mm3'] is not np.nan and isinstance(row['neurons_mm3'], (int, float)):
        row['neurons_mm3_ratio'] = row['neurons_mm3'] / mean_values['neurons_mm3']
    
    return row
    
    
# Use multiprocessing Pool to process rows in parallel
if __name__ == '__main__':
    with mp.Pool(processes=mp.cpu_count()) as pool:
        new_data = pool.map(process_row, [row for _, row in df_expected.iterrows()])
    
    # Convert the list of rows back to a DataFrame
    df_expected = pd.DataFrame(new_data)



CPU times: user 67.7 ms, sys: 875 ms, total: 943 ms
Wall time: 5.09 s


In [14]:
#import ast 
#Note that we will scale the scaled data: scaled_combined_result_dataframes
combined_result_dataframes_copy = copy.deepcopy(combined_result_dataframes)


for reg in df_expected['acronym'].values:
    concatenated_result = np.array(df_expected['acronym'].values) 
    if reg not in list(combined_result_dataframes_copy.keys()):
        print(reg, " is not a leaf region recognised by transcriptomics data, it will be split to its leaves")
        #leaves_list = df_expected[df_expected['acronym'] == reg]['children'].iat[0]
        acr_list_str = df_expected[df_expected['acronym'] == reg]['acr_list'].iat[0]
        acr_list = ast.literal_eval(acr_list_str)

        # Concatenate the array and the list to cover the full area where secondary scaling takes effect
        # We combine the leaf regions with the list of regions non leaf regions
        concatenated_result = np.concatenate((concatenated_result, acr_list))
        print(len(acr_list), len(df_expected['acronym'].values), len(concatenated_result))

STR  is not a leaf region recognised by transcriptomics data, it will be split to its leaves
30 21 51


In [15]:
len(combined_result_dataframes)

703

In [16]:
%%time

# del scaled_combined_result_dataframes
# We create the dict of dfs anew!
scaled_combined_result_dataframes = {}

#Do the 2ndary scaling: we check every transcriptomic region and scale their cell types if needed: 
for key, df in combined_result_dataframes_copy.items():
    if key in concatenated_result:
        print(key, " is a leaf region which will be scaled.")
        # We find the row where this region is listed (can be a substring if the row is not a leaf region)
        matches = df_expected.map(lambda x: key in str(x))
        rows_with_substring = df_expected[matches.any(axis=1)]
        # If the region has a non-NAN value in the ratios, we multiply that type of cells with the ratio
        if not rows_with_substring.empty and pd.notna(rows_with_substring['exc_mm3_ratio'].iloc[0]):
            mask = df.index.isin(exctypes)
            df.loc[mask, 'density_mm3'] *= rows_with_substring['exc_mm3_ratio'].iloc[0]
            scaled_combined_result_dataframes[key] = df
        #break
        if not rows_with_substring.empty and pd.notna(rows_with_substring['inh_mm3_ratio'].iloc[0]):
            mask = df.index.isin(inhtypes)
            df.loc[mask, 'density_mm3'] *= rows_with_substring['inh_mm3_ratio'].iloc[0]
            scaled_combined_result_dataframes[key] = df

        if not rows_with_substring.empty and pd.notna(rows_with_substring['neurons_mm3_ratio'].iloc[0]):
            mask = df.index.isin(neurontypes)
            df.loc[mask, 'density_mm3'] *= rows_with_substring['neurons_mm3_ratio'].iloc[0]
            scaled_combined_result_dataframes[key] = df
            #break
        #print("Deleting data for " + key)
        del key, df        
    else:
       #In this case we don't have to scale
        scaled_combined_result_dataframes[key] = df
        del key, df

AAA  is a leaf region which will be scaled.
ACB  is a leaf region which will be scaled.
BA  is a leaf region which will be scaled.
CA1slm  is a leaf region which will be scaled.
CA1so  is a leaf region which will be scaled.
CA1sp  is a leaf region which will be scaled.
CA1sr  is a leaf region which will be scaled.
CA2slm  is a leaf region which will be scaled.
CA2so  is a leaf region which will be scaled.
CA2sp  is a leaf region which will be scaled.
CA2sr  is a leaf region which will be scaled.
CA3slm  is a leaf region which will be scaled.
CA3so  is a leaf region which will be scaled.
CA3sp  is a leaf region which will be scaled.
CA3sr  is a leaf region which will be scaled.
CEAc  is a leaf region which will be scaled.
CEAl  is a leaf region which will be scaled.
CEAm  is a leaf region which will be scaled.
CP  is a leaf region which will be scaled.
FS  is a leaf region which will be scaled.
IA  is a leaf region which will be scaled.
LSc  is a leaf region which will be scaled.
LSr  i

In [17]:
len(scaled_combined_result_dataframes)

703

In [18]:
# Assuming 'total_cells' is your dictionary of DataFrames
with open(f'{root_folder}scaled_densities_local.pickle', 'wb') as f:
    pickle.dump(scaled_combined_result_dataframes, f)

print(f"Saved here: {root_folder}")

Saved here: /gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/


In [19]:
#del scaled_combined_result_dataframes

# Global scaling

This notebook starts after data was created from MERFISH slices and stored as csv and pickle files. We examine scaling individual cell groups based on literature: https://www.pnas.org/doi/full/10.1073/pnas.0604911103

Total cells (M): 108.69 ± 16.25

Total neurons (M): 70.89 ± 10.41

CCTX: 17.8 ± 3.4% of neurons (this is not the isocortex)

CRB: 59.0 ± 5.0% of neurons

"Cerebral cortex" 688 = "Cortical plate" 695 + "Cortical subplate" 703
"Cerebral nuclei is not part of the cer.ctx)
Cerebral cortex is not the Cerebellar cortex (the Cerebellar cortex is part of the Cerebellum)
From test_tutorial_cerebellum.ipynb we decided to only scale exc cells (ie granular layer) in the CRB.



In [20]:
#If we want we can load the previous step

file = os.path.join( root_folder, 'scaled_densities_local.pickle' )
#file = os.path.join( root_folder, 'non_scaled_densities_new.pickle' )

#Load region id volumes from volume_calc_from_template.ipynb
with open(file, 'rb') as pickle_file:
    scaled_combined_result_dataframes = pickle.load(pickle_file)
print("Loaded pickle file.")

Loaded pickle file.


In [21]:
#in this cell we just give ground truth cell counts from the paper
total_cells = 108.69 * 1_000_000
total_neurons = 70.89 * 1_000_000
total_ctx = total_neurons * 17.8 / 100 #17.8% of the total neurons
total_crb = total_neurons * 59.0 / 100 #59% of the total neurons
total_rest = total_neurons * ( 100 - 59.0 -17.8 )/ 100
print("Neurons only: ")
print("CTX: " '{:,.0f}'.format(total_ctx,), "CRB: " '{:,.0f}'.format(total_crb),"REST: " '{:,.0f}'.format(total_rest))

Neurons only: 
CTX: 12,618,420 CRB: 41,825,100 REST: 16,446,480


In [22]:
#Next we have to convert densities to cell counts for the scaling. This way we can measure the scaling ratio between the current densities and the desired ones. For this we need voxel size

# Edge length of the voxel in millimeters
edge_length_mm = 25.0 / 1000.0  # Convert micrometers to millimeters

# Calculate the volume of the voxel in cubic millimeters
volume25_mm3 = round(edge_length_mm ** 3, 10)

print("Volume of the voxel in mm^3:", volume25_mm3)

# Edge length of the voxel in millimeters
edge_length_mm = 10.0 / 1000.0  # Convert micrometers to millimeters

# Calculate the volume of the voxel in cubic millimeters
volume10_mm3 = round(edge_length_mm ** 3, 10)

print("Volume of the voxel in mm^3:", volume10_mm3)


Volume of the voxel in mm^3: 1.5625e-05
Volume of the voxel in mm^3: 1e-06


In [23]:
%%time 
total_cells = {}

if CCFv3.shape == (528, 320, 456):
    print("annotation vol is 25 um resolution")
    for region_name, df in scaled_combined_result_dataframes.items():
        # Get the corresponding volume for the region
        label = volumes.loc[volumes['cluster_as_filename'] == region_name, 'parcellation_label'].values[0]
        id_ = int(label.rsplit('-', 1)[-1])
        #id_ = region_map.find(region_name, attr='name', ignore_case=True, with_descendants=False)
        volume = np.count_nonzero(CCFv3 == id_) * volume25_mm3
        #print(region_name, label, id_, f"{volume:.6f}")
        # Calculate total cells by multiplying density by volume
        total_cells[region_name] = pd.DataFrame({'total_cellnumber': df['density_mm3'] * volume})

elif CCFv3.shape == (566, 320, 456):
    print("annotation vol is 25 um resolution but it is larger than the Allen Institute's CCFv3")
    for region_name, df in scaled_combined_result_dataframes.items():
        # Get the corresponding volume for the region
        label = volumes.loc[volumes['cluster_as_filename'] == region_name, 'parcellation_label'].values[0]
        id_ = int(label.rsplit('-', 1)[-1])
        #id_ = region_map.find(region_name, attr='name', ignore_case=True, with_descendants=False)
        volume = np.count_nonzero(CCFv3 == id_) * volume25_mm3
        #print(region_name, label, id_, f"{volume:.6f}")
        # Calculate total cells by multiplying density by volume
        total_cells[region_name] = pd.DataFrame({'total_cellnumber': df['density_mm3'] * volume})

elif CCFv3.shape == (1320, 800, 1140) or CCFv3.shape == (1140, 800, 1320):
    print("*** annotation vol is 10 um resolution *** ")
    for region_name, df in scaled_combined_result_dataframes.items():
        # Get the corresponding volume for the region
        label = volumes.loc[volumes['cluster_as_filename'] == region_name, 'parcellation_label'].values[0]
        id_ = int(label.rsplit('-', 1)[-1])
        #id_ = region_map.find(region_name, attr='name', ignore_case=True, with_descendants=False)
        #print(region_name, label, id_)
        
        if CCFv3.shape == (1320, 800, 1140):
            volume = np.count_nonzero(CCFv3 == id_) * volume10_mm3
        elif CCFv3.shape == (1140, 800, 1320):
            volume = np.count_nonzero(CCFv3 == id_) * volume10_mm3 
        # Calculate total cells by multiplying density by volume
        print(region_name, label, id_, f"{volume:.6f}")
        total_cells[region_name] = pd.DataFrame({'total_cellnumber': df['density_mm3'] * volume})
else:
    print("*** Annotation volume shape is not understood. *** ")

annotation vol is 25 um resolution but it is larger than the Allen Institute's CCFv3
CPU times: user 40.3 s, sys: 5.16 ms, total: 40.3 s
Wall time: 40.3 s


In [24]:
len(total_cells)

703

In [25]:
#We can save the file just in case. 

# Assuming 'total_cells' is your dictionary of DataFrames
with open(f'{root_folder}scaled_total_cells.pickle', 'wb') as f:
    pickle.dump(total_cells, f)

print(f"Saved here: {root_folder}")

Saved here: /gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/


In [26]:
#Now we need to isolate specific regions from the others to scale them differently

isocortex = (
    region_map.find("Isocortex", attr="name", with_descendants=True)
    | region_map.find("Entorhinal area", attr="name", with_descendants=True)
    | region_map.find("Piriform area", attr="name", with_descendants=True)
)
cerebellum = region_map.find(
        "Cerebellum", attr="name", with_descendants=True
    ) | region_map.find("arbor vitae", attr="name", with_descendants=True)

fiber_tracts_ids = (
    region_map.find("fiber tracts", attr="name", with_descendants=True)
    | region_map.find("grooves", attr="name", with_descendants=True)
    | region_map.find("ventricular systems", attr="name", with_descendants=True)
)
hippocampus = (
    region_map.find("Hippocampal region", attr="name", with_descendants=True)
)
hippocampal_formation = (
    region_map.find("Hippocampal formation", attr="name", with_descendants=True)
)
thalamus = (
    region_map.find("Thalamus", attr="name", with_descendants=True)
)
striatum = (
    region_map.find("Striatum", attr="name", with_descendants=True)
)
VPL = (
    region_map.find("Ventral posterolateral nucleus of the thalamus", attr="name", with_descendants=True)
)
LGd = (
    region_map.find("Dorsal part of the lateral geniculate complex", attr="name", with_descendants=True)
)
VPM = (
    region_map.find("Ventral posteromedial nucleus of the thalamus", attr="name", with_descendants=True)
)
MOB = (
    region_map.find("Main olfactory bulb", attr="name", with_descendants=True)
)

#This cell will complement the two cells above
Hindbrain = (
    region_map.find("Hindbrain", attr="name", with_descendants=True)
)
Midbrain = (
    region_map.find("Midbrain", attr="name", with_descendants=True)
)
Interbrain = (
    region_map.find("Interbrain", attr="name", with_descendants=True)
)
Olfactoryareas = (
    region_map.find("Olfactory areas", attr="name", with_descendants=True)
)
Corticalsubplate = (
    region_map.find("Cortical subplate", attr="name", with_descendants=True)
)
Cerebralnuclei = (
    region_map.find("Cerebral nuclei", attr="name", with_descendants=True)
)
cerebral_cortex = (
    region_map.find("Cerebral cortex", attr="name", with_descendants=True)
)
wholebrain = (
    region_map.find("root", attr="name", with_descendants=True)
)

#This is the final dictionary of all region names and their region ids
areas = {
    'wholebrain': wholebrain, 'Cerebral cortex': cerebral_cortex,
    'isocortex': isocortex, 'cerebellum': cerebellum, 'fiber_tracts_ids': fiber_tracts_ids, 
    'hippocampus': hippocampus, 'hippocampal_formation':hippocampal_formation, 
    'thalamus': thalamus, 'striatum': striatum, 'VPL': VPL, 'LGd': LGd, 'VPM': VPM, 'MOB': MOB,
    'Olfactory areas': Olfactoryareas, 'Cortical subplate': Corticalsubplate, 
    'Cerebral nuclei': Cerebralnuclei, 'Interbrain': Interbrain , 'Midbrain': Midbrain, 
    'Interbrain': Interbrain
}

areas_min = {
    'wholebrain': wholebrain, 'Cerebral cortex': cerebral_cortex, 'cerebellum': cerebellum,
            }

In [27]:
#We have to filter unassigned regions since we don't know enough of them.

regions = list(total_cells.keys())
print(f'Regions _covered_ in the brain / total cells dict of df: {len(regions)}')
# Assuming 'my_list' is your list
if 'unassigned' in regions:
    print("Region was found, will remove: unassigned.")
    regions.remove('unassigned')
    print("For now we are removing the region unassigned as we cannot place it in the hierarchy.json system")
    
if 'brainunassigned' in regions:
    print("Region was found, will remove brain-unassigned.")
    regions.remove('brainunassigned')
    print("For now we are removing the region brain-unassigned as we cannot place it in the hierarchy.json system")

if 'brain-unassigned' in regions:
    print("Region was found, will remove brain-unassigned.")
    regions.remove('brain-unassigned')
    print("For now we are removing the region brain-unassigned as we cannot place it in the hierarchy.json system") 
    
print(f"Remaining regions: {len(regions)}")
region_info = parcellation_annotation[parcellation_annotation['cluster_as_filename'].isin(regions)]
# region_info = parcellation_annotation[parcellation_annotation['parcellation_term_acronym'].isin(regions)]

region_info.shape

Regions _covered_ in the brain / total cells dict of df: 703
Region was found, will remove: unassigned.
For now we are removing the region unassigned as we cannot place it in the hierarchy.json system
Region was found, will remove brain-unassigned.
For now we are removing the region brain-unassigned as we cannot place it in the hierarchy.json system
Remaining regions: 701


(708, 19)

In [28]:
#Next we can calculate total cell numbers overall and in specific regions

total_cells_copy = copy.deepcopy(total_cells) 
ground_truth_cell_numbers = []

for key, df in total_cells_copy.items():
    total_cells_copy[key] = df[df.index.isin(neurontypes)]

for key, value in areas_min.items():
    
    keys_to_drop = ['unassigned', 'brain-unassigned']
    for region in regions:
        
        #filtered_data = region_info[(region_info['cluster_as_filename'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        filtered_data = parcellation_annotation[(parcellation_annotation['cluster_as_filename'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        id_ = filtered_data['label_numbers'].iloc[0]
        
        if int(id_) in value:
            #print(region, id_, "is in the set")
            None
        else:
            #print(id_, "is not in the set")
            keys_to_drop.append(region)

        #break    

    # Create a new dictionary without the specified keys (all leaf regions and their cell types in 1 region)
    total_cells_filtered = {key: value for key, value in total_cells_copy.items() if key not in keys_to_drop}

    # Concatenate all DataFrames into a single DataFrame
    concatenated_df = pd.concat(total_cells_filtered.values())

    # Group by all columns except the 'total_cells' column and sum the values
    summed_df = concatenated_df.groupby(concatenated_df.index).sum()
    print(f"The brain area {key} has {summed_df.shape[0]} different cell types")
    #print("Total sum of cells:", '{:,.0f}'.format(summed_df['density_mm3'].sum()))
    print("Total sum of neurons:", '{:,.0f}'.format(summed_df['total_cellnumber'].sum()))
    ground_truth_cell_numbers.append(summed_df['total_cellnumber'].sum())

The brain area wholebrain has 5156 different cell types
Total sum of neurons: 53,360,742
The brain area Cerebral cortex has 1800 different cell types
Total sum of neurons: 33,502,874
The brain area cerebellum has 1116 different cell types
Total sum of neurons: 3,939,848


In [31]:
#WE can store the values in a variable and get the scaling ratios
mfish_ctx = ground_truth_cell_numbers[1]
mfish_crb = ground_truth_cell_numbers[2]
mfish_rest = ground_truth_cell_numbers[0] - mfish_ctx - mfish_crb

In [32]:
#Scaling factors as percentage
1/(mfish_ctx/total_ctx)*100, 1/(mfish_crb/total_crb)*100, 1/(mfish_rest/total_rest)*100

(37.66369430556719, 1061.5918036629878, 103.31988184651408)

In [33]:
mfish_ctx, total_ctx, 1/(mfish_ctx/total_ctx), mfish_ctx*(1/(mfish_ctx/total_ctx))

(33502873.875372417, 12618420.0, 0.3766369430556719, 12618420.0)

In [34]:
#Sanity check
#no CTX or CRB keys should be printed. Only regions with multiple solutions, which is taken care of elsewhere

ctx_keys = []
crb_keys = []
else_keys = []
for key in combined_result_dataframes.keys():
    cluster_as_filename = parcellation_annotation[parcellation_annotation['cluster_as_filename'] == key]
    cluster = cluster_as_filename[cluster_as_filename['parcellation_term_set_name'] == 'substructure']
    #This != part is for ambiguous regions:
    if cluster.shape[0] != 1:
        print( key, cluster['parcellation_term_name'].values[0] )
        for key_name in cluster['cluster_as_filename'].values:
            # Add the element to else_keys list
            else_keys.append(key_name)
    elif cluster['label_numbers'].iloc[0] in cerebral_cortex:
        ctx_keys.append(cluster['cluster_as_filename'].iloc[0])
    elif cluster['label_numbers'].iloc[0] in cerebellum:
        crb_keys.append(cluster['cluster_as_filename'].iloc[0])
    else:
        else_keys.append(cluster['cluster_as_filename'].iloc[0])

#remove duplicates if there's any (in else there is)
else_keys = list(dict.fromkeys(else_keys))
crb_keys = list(dict.fromkeys(crb_keys))
ctx_keys = list(dict.fromkeys(ctx_keys))

print(len(ctx_keys), len(crb_keys), len(else_keys), )

fa corpus callosum, anterior forceps
icp inferior cerebellar peduncle
sV sensory root of the trigeminal nerve
scp superior cerebelar peduncles
st stria terminalis
309 54 340


Scaling: We will scale the regions one by one according to their anatomical positions:

- If the region was scaled earlier in the previous section we should not scale it anymore
- In the CTX we can scale with the cortical scaling ratio
- IN the CRB with the cerebellar scaling ratio
- Everywhere else with the residual scaling ratio
- Glia can be scaled by 1x (change)

In [35]:
len(scaled_combined_result_dataframes)

703

In [36]:
%%time

#Scaling will take place
combined_result_dataframes_copy = copy.deepcopy(scaled_combined_result_dataframes)
# del scaled_combined_result_dataframes
globally_scaled_combined_result_dfs = {}

#Do the scaling: CTX: neurons only, CRB: EXC only, ELSE: neuron only
for key, df in combined_result_dataframes_copy.items():
    #Leave those regions which we have scaled already as they are
    if key in concatenated_result:
        globally_scaled_combined_result_dfs[key] = df
       
    #Scale only the neurons and glia:
    elif key in ctx_keys:
        # Select rows where the value is in neurontypes
        mask = df.index.isin(neurontypes)
        
        # Divide only selected rows by scaling factor for CTX Neuron
        df.loc[mask, 'density_mm3'] /= (mfish_ctx/total_ctx)
        globally_scaled_combined_result_dfs[key] = df
        del df, mask
        
    elif key in crb_keys:
        # Select rows where the value is in  exc types
        mask = df.index.isin(exctypes)
        # Select rows where the value is in neuron+glia types (this was an attempt to see what glia numbers would be if changed)
        # combined = np.concatenate((exctypes, glia))
        # mask = df.index.isin(combined)
        
        # Divide only selected rows by scaling factor for CB EXC
        df.loc[mask, 'density_mm3'] /= (mfish_crb/total_crb)
        globally_scaled_combined_result_dfs[key] = df        
        #print(df)
        del df, mask
    
    elif key in else_keys:
        # Select rows where the value is in neurontypes
        mask = df.index.isin(neurontypes)
        
        # Divide only selected rows by scaling factor for REST
        df.loc[mask, 'density_mm3'] /= (mfish_rest/total_rest)
        globally_scaled_combined_result_dfs[key] = df        
        del df, mask

    else:
        print(key, "is an Issue!")


# # Check if multiplication took place
# for key, df in combined_result_dataframes_copy.items():
#     if key in ctx_keys:
#         print(f'Original value for key {key}:')
#         print(df['density_mm3'])
        
# for key, df in scaled_combined_result_dataframes.items():
#     if key in ctx_keys:
#         print(f'Copied value for key {key}:')
#         print(df['density_mm3'])

#del combined_result_dataframes_copy

CPU times: user 559 ms, sys: 9 µs, total: 559 ms
Wall time: 558 ms


In [37]:
len(globally_scaled_combined_result_dfs)

703

# Validation

In [38]:
import numpy as np
from voxcell import VoxelData
from voxcell.nexus.voxelbrain import RegionMap
from argparse import ArgumentParser
import os, sys
import warnings
import inspect

# Parse arguments

parser = ArgumentParser()
parser.add_argument("annotation")
parser.add_argument("hierarchy")
parser.add_argument("density_folder")
parser.add_argument("cell_density")
parser.add_argument("inhibitory_density_folder")
parser.add_argument("excitatory_ME_types_folder")
parser.add_argument("inhibitory_ME_types_folder")
parser.add_argument("excitatory_ME_types_transplant_folder")
parser.add_argument("inhibitory_ME_types_transplant_folder")

#brain_regions = '/gpfs/bbp.cscs.ch/home/veraszto/bbp_prod_files/2022/annotation_25.nrrd'
brain_regions = '/home/piluso/data/00_allen_brain_atlas/ccfv2/annotation_25_ccfv2_combined.nrrd '

#hierarchy_json = '/gpfs/bbp.cscs.ch/home/veraszto/bbp_prod_files/1.json'
hierarchy_json ='/gpfs/bbp.cscs.ch/data/project/proj84/atlas_pipeline_runs/2024-05-15T22:44:26+02:00/hierarchy_ccfv3_l23split_barrelsplit.json'
#cell_density = '/gpfs/bbp.cscs.ch/home/veraszto/leaves_only/cell_densities_correctednissl/neuron_density.nrrd'
cell_density = '/gpfs/bbp.cscs.ch/home/piluso/cell_atlas/03_warped_annotation_fix_last/blue_brain_atlas_pipeline/leaves_only/overall_cell_density_correctednissl_validated.nrrd'
#cell_density = '/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/total_celltypes_densities.nrrd'
density_folder = '/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/'

In [39]:
print("\nRegion filters setting")
region_map = RegionMap.load_json(hierarchy_json)

cerebellum = region_map.find(
        "Cerebellum", attr="name", with_descendants=True
    ) | region_map.find("arbor vitae", attr="name", with_descendants=True)
isocortex = (
    region_map.find("Isocortex", attr="name", with_descendants=True)
    | region_map.find("Entorhinal area", attr="name", with_descendants=True)
    | region_map.find("Piriform area", attr="name", with_descendants=True)
)
fiber_tracts_ids = (
    region_map.find("fiber tracts", attr="name", with_descendants=True)
    | region_map.find("grooves", attr="name", with_descendants=True)
    | region_map.find("ventricular systems", attr="name", with_descendants=True)
)
hippocampus = (
    region_map.find("Hippocampal region", attr="name", with_descendants=True)
)
hippocampal_formation = (
    region_map.find("Hippocampal formation", attr="name", with_descendants=True)
)
thalamus = (
    region_map.find("Thalamus", attr="name", with_descendants=True)
)
striatum = (
    region_map.find("Striatum", attr="name", with_descendants=True)
)
VPL = (
    region_map.find("Ventral posterolateral nucleus of the thalamus", attr="name", with_descendants=True)
)
LGd = (
    region_map.find("Dorsal part of the lateral geniculate complex", attr="name", with_descendants=True)
)
VPM = (
    region_map.find("Ventral posteromedial nucleus of the thalamus", attr="name", with_descendants=True)
)
MOB = (
    region_map.find("Main olfactory bulb", attr="name", with_descendants=True)
)

if cerebellum == [] or \
isocortex == [] or \
fiber_tracts_ids == [] or \
hippocampus == [] or \
striatum == []:
    raise ValueError("ERROR: some region filters return empty sets")

print("Done")

annotation = VoxelData.load_nrrd('/gpfs/bbp.cscs.ch/data/project/proj84/piluso/share/general/warped_augmented_CCFv3/annotation_25_2022_CCFv3a.nrrd').raw


Region filters setting
Done


In [40]:
# # Literature values and initialization of paramters (to be moved into a configuration file)
print("\nLiterature values setting...")
voxel_volume = (25 * 1.0e-3)**3
nb_voxels_ccfv2_2011 = 31984720 # 499.8 mm^3
nb_voxels_ccfv2_2015 = 31992356 # = 499.9 mm^3
nb_voxels_ccfv2_2015_fiber_tracts = 3525860 # = 55.1 mm^3
nb_voxels_ccfv2_2015_whithout_fiber_tracts = nb_voxels_ccfv2_2015 - nb_voxels_ccfv2_2015_fiber_tracts # 444.8 mm^3
nb_voxels_ccfv3_2015 = 32387385 # = 506.1 mm^3
nb_voxels_ccfv3_2022 = 32261391 # = 504.1 mm^3
nb_voxels_ccfv3_2022_fiber_tracts = 2980469 # = 46.6 mm^3
nb_voxels_ccfv2_2022_whithout_fiber_tracts = nb_voxels_ccfv3_2022 - nb_voxels_ccfv3_2022_fiber_tracts # 457.5 mm^3
nb_voxels_ccfv3_2022_augmented = None # Not yet available
default_neuron_proportion = 0.16 # Default neuron density tolerance inheritated from total neuron tolerance when no data is available
default_glia_proportion = 0.20 # Default glia density tolerance inheritated from total glia tolerance when no data is available
default_cell_proportion = 0.18 # Default cell density tolerance inheritated from total cell tolerance when no data is available

# Literature values for the whole brain (to be moved into a configuration file)
wh_mouse_brain_vol_litt_m = 32570240 # = 508.91 mm^3 in Badea et al., 2007
wh_mouse_brain_vol_tolerance_m = 1498880 # = 23.42 mm^3 (5%) in Badea et al., 2007wh_mouse_brain_vol_litt_m = 32570240 # = 508.91 mm^3 in Badea et al., 2007
wh_mouse_brain_vol_litt = 508.91 # = 508.91 mm^3 in Badea et al., 2007
wh_mouse_brain_vol_tolerance = 23.42 # = 23.42 mm^3 (5%) in Badea et al., 2007
neuron_dens_fiber_tracts_litt = 0 # Rodarie et al., 2022
neuron_dens_fiber_tracts_tolerance = 0 # Rodarie et al., 2022
neuron_dens_litt = 71760000/wh_mouse_brain_vol_litt # = 67,870,000 + 3,890,000 Table 1 in Herculano-Houzel et al., 2011
neuron_dens_tolerance = 11660000/wh_mouse_brain_vol_litt # = 10,410,000 + 1,250,000 (16%) Table 1 in Herculano-Houzel et al., 2011
glia_dens_litt = 39320000/wh_mouse_brain_vol_litt # = 33,860,000 + 5,460,000 Table 1 in Herculano-Houzel et al., 2011
glia_dens_tolerance = 7810000/wh_mouse_brain_vol_litt # = 6,660,000 + 1,150,000 (20%) Table 1 in Herculano-Houzel et al., 2011
cell_dens_litt = 111080000/wh_mouse_brain_vol_litt # = 71,760,000 + 39,320,000 Table 1 in Herculano-Houzel et al., 2011 after summing neuron + glia
cell_dens_tolerance = 19470000/wh_mouse_brain_vol_litt # = 11,660,000 + 7,810,000 (18%) Table 1 in Herculano-Houzel et al., 2011 after summing neuron + glia
astrocyte_dens_litt = None # Not available
astrocyte_dens_tolerance = None # Not available
microglia_dens_litt = None # Not available
microglia_dens_tolerance = None # Not available
oligodendrocyte_dens_litt = None # Not available
oligodendrocyte_dens_tolerance = None # Not available
inhibitory_neuron_dens_litt = 14550000/wh_mouse_brain_vol_litt # Table 3 in Rodarie et al., 2022
inhibitory_neuron_dens_tolerance = round(inhibitory_neuron_dens_litt * default_neuron_proportion) # default Not available /!\
excitatory_neurons_dens_litt = None # Not available
excitatory_neurons_dens_tolerance = None # Not available
pv_dens_litt = 2631372/wh_mouse_brain_vol_litt # = 5916 * 445 in measurements.csv (~17.6% * Inhibitory in Table 3) in Rodarie et al., 2022
pv_dens_tolerance = 237072/wh_mouse_brain_vol_litt # = 533 (9%) in measurements.csv in Rodarie et al., 2022
sst_dens_litt = 2253658/wh_mouse_brain_vol_litt# = 5067 * 445 (~15.8% * Inhibitory in Table 3) in Rodarie et al., 2022
sst_dens_tolerance = 234498/wh_mouse_brain_vol_litt # = 527 (10%) in measurements.csv in Rodarie et al., 2022
vip_dens_litt = 434928/wh_mouse_brain_vol_litt # = 978 * 445 (~3.1% * Inhibitory in Table 3) in Rodarie et al., 2022
vip_dens_tolerance = 30616/wh_mouse_brain_vol_litt # 69 (7%) in measurements.csv in Rodarie et al., 2022
rest_inhi_dens_litt = (inhibitory_neuron_dens_litt - (pv_dens_litt + sst_dens_litt + vip_dens_litt))/wh_mouse_brain_vol_litt # (~63.5% * Inhibitory in Table 3) in Rodarie et al., 2022
rest_inhi_dens_tolerance = (inhibitory_neuron_dens_tolerance - (pv_dens_tolerance + sst_dens_tolerance + vip_dens_tolerance))/wh_mouse_brain_vol_litt # Rodarie et al., 2022

# Region support in number of voxels
isocortex_nb_vox = len(np.where(np.isin(annotation, list(isocortex)) != 0)[0])
hippocampus_nb_vox = len(np.where(np.isin(annotation, list(hippocampus)) != 0)[0])
striatum_nb_vox = len(np.where(np.isin(annotation, list(striatum)) != 0)[0])
thalamus_nb_vox = len(np.where(np.isin(annotation, list(thalamus)) != 0)[0])
VPL_nb_vox = len(np.where(np.isin(annotation, list(VPL)) != 0)[0])
LGd_nb_vox = len(np.where(np.isin(annotation, list(LGd)) != 0)[0])
VPM_nb_vox = len(np.where(np.isin(annotation, list(VPM)) != 0)[0])
MOB_nb_vox = len(np.where(np.isin(annotation, list(MOB)) != 0)[0])
hippocampal_formation_nb_vox = len(np.where(np.isin(annotation, list(hippocampal_formation)) != 0)[0])

# Literature values for sub-regions (to be moved into a configuration file)
isocortex_neuron_dens_litt = 2*5048837/(isocortex_nb_vox * voxel_volume) # Table 1 in Herculano_Houzel et al., 2013
isocortex_neuron_dens_tolerance = 2*412123/(isocortex_nb_vox * voxel_volume) # Table 1 in Herculano_Houzel et al., 2013
isocortex_glia_dens_litt = 2*6640234/(isocortex_nb_vox * voxel_volume) # Table 1 in Herculano_Houzel et al., 2013
isocortex_glia_dens_tolerance = 2*244643/(isocortex_nb_vox * voxel_volume) # Table 1 in Herculano_Houzel et al., 2013
isocortex_cell_dens_litt = (isocortex_neuron_dens_litt + isocortex_glia_dens_litt) # Table 1 in Herculano_Houzel et al., 2013
isocortex_cell_dens_tolerance = (isocortex_neuron_dens_tolerance + isocortex_glia_dens_tolerance) # Table 1 in Herculano_Houzel et al., 2013
isocortex_oligo_dens_litt = 12500 # Table 1 in Erö et al., 2018
isocortex_oligo_dens_tolerance = round(isocortex_oligo_dens_litt * default_glia_proportion) # default Not available /!\
isocortex_astro_dens_litt = 15696 # Table 1 in Erö et al., 2018
isocortex_astro_dens_tolerance = round(isocortex_astro_dens_litt * default_glia_proportion) # default Not available /!\
isocortex_microglia_dens_litt = 6500 # Table 1 in Erö et al., 2018
isocortex_microglia_dens_tolerance = round(isocortex_microglia_dens_litt * default_glia_proportion) # default Not available /!\
cerebellum_volume = 59.65 # mm^3 from Table 1 in Zhang et al., 2011
cerebellum_volume_torlerance = 3.65 # mm^3 from Table 1 in Zhang et al., 2011
cerebellum_neuron_dens_litt = 42220000/cerebellum_volume # Table 1 in Herculano_Houzel et al., 2011
cerebellum_neuron_dens_tolerance = 9280000/cerebellum_volume # Table 1 in Herculano_Houzel et al., 2011
cerebellum_glia_dens_litt = 6950000/cerebellum_volume # Table 1 in Herculano_Houzel et al., 2011
cerebellum_glia_dens_tolerance = 1500000/cerebellum_volume # Table 1 in Herculano_Houzel et al., 2011
cerebellum_cell_dens_litt = (cerebellum_neuron_dens_litt + cerebellum_glia_dens_litt) # Table 1 in Herculano_Houzel et al., 2011
cerebellum_cell_dens_tolerance = (cerebellum_neuron_dens_tolerance + cerebellum_glia_dens_tolerance) # Table 1 in Herculano_Houzel et al., 2011
cerebellum_oligo_dens_litt = 13750 # = Table 1 in Erö et al., 2018
cerebellum_oligo_dens_tolerance = 1768 # Table 1 in Erö et al., 2018
cerebellum_astro_dens_litt = 1512 # Table 1 in Erö et al., 2018
cerebellum_astro_dens_tolerance = round(cerebellum_astro_dens_litt * default_glia_proportion) # default Not available /!\
cerebellum_microglia_dens_litt = 8624 # Table 1 in Erö et al., 2018
cerebellum_microglia_dens_tolerance = 659 # Table 1 in Erö et al., 2018
hippocampus_neuron_dens_litt = 20848 # from Table 3 in Keller et al., 2018
hippocampus_neuron_dens_tolerance = hippocampus_neuron_dens_litt * default_neuron_proportion # default Not available /!\
hippocampus_oligo_dens_litt = 9425 # Table 1 in Erö et al., 2018
hippocampus_oligo_dens_tolerance = round(hippocampus_oligo_dens_litt * default_glia_proportion) # default Not available /!\
hippocampus_astro_dens_litt = 16737 # Table 1 in Erö et al., 2018
hippocampus_astro_dens_tolerance = 10496 # Table 1 in Erö et al., 2018
hippocampus_microglia_dens_litt = 3248 # Table 1 in Erö et al., 2018
hippocampus_microglia_dens_tolerance = 1563 # Table 1 in Erö et al., 2018
# striatum_neuron_dens_litt = 802679/(striatum_nb_vox * voxel_volume)  # Andsberg et al., 2001
# striatum_neuron_dens_tolerance = 31665/(striatum_nb_vox * voxel_volume)# Andsberg et al., 2001
striatum_neuron_dens_litt = 120110 # from Table 5 in Keller et al., 2018
striatum_neuron_dens_tolerance = 31800 # from Table 5 in Keller et al., 2018
striatum_oligo_dens_litt = 9950 # Table 1 in Erö et al., 2018
striatum_oligo_dens_tolerance = 4036 # Table 1 in Erö et al., 2018
striatum_astro_dens_litt = 9867 # Table 1 in Erö et al., 2018
striatum_astro_dens_tolerance = 5547 # Table 1 in Erö et al., 2018
striatum_microglia_dens_litt = 12101 # Table 1 in Erö et al., 2018
striatum_microglia_dens_tolerance = 1930 # Table 1 in Erö et al., 2018
LGd_neuron_dens_litt = 141000 # from Table 5 in Keller et al., 2018
LGd_neuron_dens_tolerance = 24000 # from Table 5 in Keller et al., 2018
thalamus_glia_dens_litt = 80966 # from Table 5 in Keller et al., 2018
thalamus_glia_dens_tolerance = thalamus_glia_dens_litt * default_glia_proportion # default Not available /!\
thalamus_cell_dens_litt = 122450 # from Table 5 in Keller et al., 2018
thalamus_cell_dens_tolerance = thalamus_cell_dens_litt * default_cell_proportion # default Not available /!\
VPM_neuron_dens_litt = 83100 # from Table 5 in Keller et al., 2018
VPM_neuron_dens_tolerance = 7900 # from Table 5 in Keller et al., 2018
VPL_neuron_dens_litt = 57466.92 # LNMC data from 2019 Thalamic Release Report pages 21-23 https://docs.google.com/document/d/1maQ8VIwaFeyOQpfUy6PoLcamp48NW9NUy6BPN_oLRQk/edit#heading=h.otpvbup5qa45
# VPL_neuron_dens_tolerance = 5201.4 # LNMC data from 2019 Thalamic Release Report pages 21-23 https://docs.google.com/document/d/1maQ8VIwaFeyOQpfUy6PoLcamp48NW9NUy6BPN_oLRQk/edit#heading=h.otpvbup5qa45
VPL_neuron_dens_tolerance = VPL_neuron_dens_litt * default_neuron_proportion # default Not available /!\
VPL_pv_dens_litt = 1238.528635	# from Kim et al., 2017
VPL_pv_dens_tolerance = 575.6900057 # from Kim et al., 2017
VPL_sst_dens_litt = 1609.664213 # from Kim et al., 2017
VPL_sst_dens_tolerance = 1186.589608 # from Kim et al., 2017
VPL_vip_dens_litt = 0.746468649 # from Kim et al., 2017
VPL_vip_dens_tolerance = 1.296087826 # from Kim et al., 2017
MOB_cell_dens_litt = 383148 # from Tables in Parrish-Aungst et al., 2007
MOB_cell_dens_tolerance = 27027 # from Tables in Parrish-Aungst et al., 2007
MOB_neuron_dens_litt = 246422 # from Tables in Parrish-Aungst et al., 2007
MOB_neuron_dens_tolerance = 17488 # from Tables in Parrish-Aungst et al., 2007
MOB_glia_dens_litt = 136725 # from Tables in Parrish-Aungst et al., 2007
MOB_glia_dens_tolerance = 9539 # from Tables in Parrish-Aungst et al., 2007

print("Done")


Literature values setting...
Done


In [41]:
print("\nLaunching the assertions and writing ouptut result in the log file...")

# with open(output_log_file_path, "a") as log_file:

# Catching all prints to write them into a log file
# sys.stdout = log_file

# Functions
def print_range_bar(value, min_value, max_value, bar_length=40):
    """
    Printing the range bar of the given density compared to literature plus some statisics elements (std, z-score)
    @method print_range_bar
    @param {Float} value The input dentity value to assert
    @param {Float} min_value The min density value set by literature
    @param {Float} max_value The max density value set by literature
    @param {Integer} bar_length The length of the bar to plot
    @return {None}
    """
    progress = (value - min_value) / (max_value - min_value)
    progress = max(0, min(1, progress))
    arrow = ' ' * int(round(bar_length * progress)) + '*'
    spaces = ' ' * (bar_length - len(arrow))
    # Format the values as integers and display without decimal places
    value = int(round(value))
    min_value = int(round(min_value))
    max_value = int(round(max_value))
    mean_val = (min_value + max_value)/2
    std = mean_val - min_value
    z_score = round((value - mean_val)/std, 2)
    std_percentage = round((mean_val - min_value)/mean_val*100, 1)
    range_to_print = f'Range: [{arrow}{spaces}] {value}  | z-score = {z_score} |  -{std_percentage}% [{min_value}, {max_value}] +{std_percentage}%'
    if value < min_value:
        range_to_print = range_to_print.replace("[*", "*[")
        print(range_to_print)
        # try:
        #     print("diff = " + str(mean_val - value) + "  |   -" + str(round(mean_val - value)/mean_val*100,1) + "%")
        # except:
        #     print("EXCEPTION: low limit")
        #     pass
    elif value > max_value:
        # try:
        #     print("diff = " + str(mean_val - value) + "  |   +" + str(round(mean_val - value)/mean_val*100,1) + "%")
        # except:
        #     print("EXCEPTION: high limit")
        #     pass
        range_to_print = range_to_print.replace("*]", "]*")
        print(range_to_print)
    else:
        print(range_to_print)
    return


class DensityError(Exception):
    def __init__(self, message):
        self.message = message
        super().__init__(self.message)


def z_score_assertion(value = 0, min_value = 0, max_value = 0, assertion_message = ""):
    """
    Asserting the z-score for a given assertion is in the right range from literature:
        - |z| <= 1: VALIDATED
        - 1 < |z| <= 2: WARNING
        - |z| > 2: ERROR
    @method z_score_assertion
    @param {Float} value The input dentity value to assert
    @param {Float} min_value The min density value set by literature
    @param {Float} max_value The max density value set by literature
    @param {String} assertion_message The assertion message to print if a Warning or an Error is raised
    @return {None}
    """
    mean_val = (min_value + max_value)/2
    std = mean_val - min_value
    z_score = round((value - mean_val)/std, 2)
    if abs(z_score) <= 1:
        print("Validated")
    elif (abs(z_score) > 1) and (abs(z_score) <= 2):
        warnings.warn(assertion_message, UserWarning)
        print("WARNING:", assertion_message)
    elif abs(z_score) > 2:
        print("ERROR:", assertion_message)
        #raise DensityError(assertion_message)
    else:
        raise ValueError("Uknown value")
    return


def z_score_assertion_sub_regions(value = 0, min_value = 0, max_value = 0, assertion_message = ""):
    """
    Asserting the z-score for a given assertion is in the right range from literature:
        - |z| <= 1: VALIDATED
        - |z| > 1: WARNING
    @method z_score_assertion_sub_regions
    @param {Float} value The input dentity value to assert
    @param {Float} min_value The min density value set by literature
    @param {Float} max_value The max density value set by literature
    @param {String} assertion_message The assertion message to print if a Warning is raised
    @return {None}
    """
    mean_val = (min_value + max_value)/2
    std = mean_val - min_value
    z_score = round((value - mean_val)/std, 2)
    if abs(z_score) <= 1:
        print("Validated")
    elif abs(z_score) > 1:
        warnings.warn(assertion_message, UserWarning)
        print("WARNING:", assertion_message)
    else:
        raise ValueError("Uknown value")
    return



def z_score_assertion_after_transplant(value = 0, min_value = 0, max_value = 0, assertion_message = ""):
    """
    Asserting the z-score for a given assertion after transplant is in the right range from literature:
        - |z| <= 1: VALIDATED
        - |z| > 1: ERROR
    @method z_score_assertion
    @param {Float} value The input dentity value to assert
    @param {Float} min_value The min density value set by literature
    @param {Float} max_value The max density value set by literature
    @param {String} assertion_message The assertion message to print if a Warning or an Error is raised
    @return {None}
    """
    mean_val = (min_value + max_value)/2
    std = mean_val - min_value
    z_score = round((value - mean_val)/std, 2)
    if abs(z_score) <= 1:
        print("Validated")
    elif (abs(z_score) > 1):
        print("ERROR:", assertion_message)
        #raise DensityError(assertion_message)
    else:
        raise ValueError("Uknown value")
    return

# =============================================================================================




Launching the assertions and writing ouptut result in the log file...


In [42]:
# Assertion on the total volumetry of the annotation
whole_brain_annotation_dens = len(np.where(annotation != 0)[0])
annotation_dens_diff = abs(wh_mouse_brain_vol_litt_m - whole_brain_annotation_dens)
print("\nAssertion on total annotation volumetry (mm^3)")
print_range_bar(whole_brain_annotation_dens, wh_mouse_brain_vol_litt_m - wh_mouse_brain_vol_tolerance_m, wh_mouse_brain_vol_litt_m + wh_mouse_brain_vol_tolerance_m)
assertion_message = "total annotation volumetry out of literature range"
z_score_assertion(whole_brain_annotation_dens, wh_mouse_brain_vol_litt_m - wh_mouse_brain_vol_tolerance_m, wh_mouse_brain_vol_litt_m + wh_mouse_brain_vol_tolerance_m, assertion_message)



Assertion on total annotation volumetry (mm^3)
Range: [                      *                 ] 32750610  | z-score = 0.12 |  -4.6% [31071360, 34069120] +4.6%
Validated


In [40]:
from helper_functions import create_combined_dataframe
import matplotlib.pyplot as plt

In [51]:
sc_f_inh = 1.9
sc_f_glia = 0.75
sc_f_exc = 0.75

#0.5, 0.6, 0.7, 0.8, 0.9, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6,
for sc_factor in [0.5, 0.6, 0.7, 0.8, 0.9, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6,1.7, 1.8, 1.9]:
    #Scaling will take place
    combined_result_dataframes_copy = copy.deepcopy(globally_scaled_combined_result_dfs)
    # del scaled_combined_result_dataframes
    globally_scaled_combined_result_dfs_2 = {}
    
    #Do the scaling:
    for key, df in combined_result_dataframes_copy.items():
        #Leave those regions which we have overwritten as they are
        if key in concatenated_result:       
    
            # Select rows where the value is in glia
            mask = df.index.isin(glia)
            
            # Divide only selected rows by scaling factor for CTX
            df.loc[mask, 'density_mm3'] /= sc_factor #Change later
            globally_scaled_combined_result_dfs_2[key] = df
            
        #Scale CORTEX:
        elif key in ctx_keys:     
            # Select rows where the value is in excitatory neurons
            mask = df.index.isin(exctypes)
            
            # Divide only selected rows by scaling factor for CTX GLIA
            df.loc[mask, 'density_mm3'] /= sc_f_exc #Change later
            globally_scaled_combined_result_dfs_2[key] = df
            del mask
            
            # Select rows where the value is in glia
            mask = df.index.isin(glia)
            
            # Divide only selected rows by scaling factor for CTX GLIA
            df.loc[mask, 'density_mm3'] /= sc_factor #Change later
            globally_scaled_combined_result_dfs_2[key] = df
            del mask
    
            # Select rows where the value is in  inh types
            mask = df.index.isin(inhtypes)
            
            # Divide only selected rows by scaling factor for CTX INH
            df.loc[mask, 'density_mm3'] /= sc_f_inh
            globally_scaled_combined_result_dfs_2[key] = df        
            #print(df)
            del df, mask

        # Scale CEREBELLUM
        elif key in crb_keys:
            # Select rows where the value is in excitatory neurons
            mask = df.index.isin(exctypes)
            
            # Divide only selected rows by scaling factor for CTX EXC
            df.loc[mask, 'density_mm3'] /= sc_f_exc #Change later
            globally_scaled_combined_result_dfs_2[key] = df
            del mask
            
            # Select rows where the value is in glia
            mask = df.index.isin(glia)
            
            # Divide only selected rows by scaling factor for CB GLIA
            df.loc[mask, 'density_mm3'] /= sc_factor #Change later
            globally_scaled_combined_result_dfs_2[key] = df
            del mask
    
            # Select rows where the value is in  inh types
            mask = df.index.isin(inhtypes)
            
            # Divide only selected rows by scaling factor for CB INH
            df.loc[mask, 'density_mm3'] /= sc_f_inh
            globally_scaled_combined_result_dfs_2[key] = df        
            #print(df)
            del df, mask

        # Scale REST of the BRAIN
        elif key in else_keys:
            # Select rows where the value is in excitatory neurons
            mask = df.index.isin(exctypes)
            
            # Divide only selected rows by scaling factor for CTX EXC
            df.loc[mask, 'density_mm3'] /= sc_f_exc #Change later
            globally_scaled_combined_result_dfs_2[key] = df
            del mask
            
            # Select rows where the value is in glia
            mask = df.index.isin(glia)
            
            # Divide only selected rows by scaling factor for REST GLIA
            df.loc[mask, 'density_mm3'] /= sc_factor #Change later
            globally_scaled_combined_result_dfs_2[key] = df
            del mask
    
            # Select rows where the value is in  inh types
            mask = df.index.isin(inhtypes)
            
            # Divide only selected rows by scaling factor for REST INH
            df.loc[mask, 'density_mm3'] /= sc_f_inh
            globally_scaled_combined_result_dfs_2[key] = df        
            #print(df)
            del df, mask
    
        else:
            print(key, "is an Issue!")

    # Assuming 'total_cells' is your dictionary of DataFrames
    with open(f'{root_folder}scaled_densities_new_{sc_factor}.pickle', 'wb') as f:
        pickle.dump(globally_scaled_combined_result_dfs_2, f)
    print(f"{sc_factor} factor is saved here: {root_folder}")
  

0.5 factor is saved here: /gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations_3/
0.6 factor is saved here: /gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations_3/
0.7 factor is saved here: /gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations_3/
0.8 factor is saved here: /gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations_3/
0.9 factor is saved here: /gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations_3/
1.1 factor is saved here: /gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations_3/
1.2 factor is saved here: /gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations_3/
1.3 factor is saved here: /gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/

In [52]:
def process_type(types, file_name):
    # Filter DataFrames based on types
    filtered_dataframes = {key: value for key, value in shuffled_combined_dataframes.items() if key in types}
    
    # Combine filtered DataFrames
    combined_df = pd.concat(filtered_dataframes.values())
    
    # Sum the combined DataFrame by index
    summed_df = combined_df.groupby(combined_df.index).sum()
    
    # Validate result
    result = nrrd_for_validation(summed_df, parcellation_annotation, CCFv3)
    
    # Clean up
    del combined_df, summed_df, filtered_dataframes
    
    return (result, file_name)

def main():
    # Define the parameters for each process
    tasks = [
        # (neurontypes, "scaled_total_neuron_densities"),
        # (nonneurontypes, "scaled_total_nonneuron_densities"),
        # (exctypes, "scaled_total_excitatory_densities"),
        # (inhtypes, "scaled_total_inhibitory_densities"),
        # (astrotypes, "scaled_total_astrotypes_densities"),
        # (microglia, "scaled_total_microglia_densities"),
        # (oligos, "scaled_total_oligocyte_densities"),
        (glia, "scaled_total_glia_densities"),
        # (exci_inhib_sum, "scaled_total_excinh_densities"),
        # (celltypes, "scaled_total_celltypes_densities"),
    ]    
    # Create a multiprocessing Pool
    with mp.Pool(processes=mp.cpu_count()) as pool:
        results = pool.starmap(process_type, tasks)
    
    # Sequentially write the .nrrd files to avoid concurrent writes
    for result, file_name in results:
        print(file_name)
        nrrd.write(f"{root_folder}{file_name}.nrrd", result)

EXC VALIDATION WITH NEURONS

In [50]:
%%time

from contextlib import redirect_stdout
log_file_path = f"{root_folder}/log/EXC_print_messages.txt"
with open(log_file_path, "w") as log_file:
    with redirect_stdout(log_file):
        for sc_factor in [0.5, 0.6, 0.7, 0.8, 0.9, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6,1.7, 1.8, 1.9]: 
            try: del shuffled_combined_dataframes
            except NameError: pass
            
            file = os.path.join(f'{root_folder}scaled_densities_new_{sc_factor}.pickle')
            #Load region id volumes from volume_calc_from_template.ipynb
            with open(file, 'rb') as pickle_file:
                scaled_combined_result_dataframes = pickle.load(pickle_file)
        
            print("", file=log_file)
            print(f"Loaded pickle file for {sc_factor}.", file=log_file)
            log_file.flush()
            shuffled_combined_dataframes = create_combined_dataframe(scaled_combined_result_dataframes) #[celltypes]region
            print("Celltype/region dict is created.", file=log_file)
            log_file.flush()
            main()
            print("Total EXC files for sc_factor is created.", file=log_file)
            #glia_total, __ = nrrd.read(os.path.join(density_folder, "scaled_total_glia_densities.nrrd"))
            neuron, __ = nrrd.read(os.path.join(density_folder, "scaled_total_neuron_densities.nrrd"))
            print("Total EXC file loaded.", file=log_file)
            log_file.flush()

            # Assertion on total neuron densities
            neuron_dens = np.sum(neuron) / whole_brain_annotation_dens # * voxel_volume
            neuron_dens_diff = abs(neuron_dens_litt - neuron_dens)
            print("\nAssertion on total neuron densities (/mm^3)")
            print_range_bar(neuron_dens, neuron_dens_litt - neuron_dens_tolerance, neuron_dens_litt + neuron_dens_tolerance)
            assertion_message = "total neuron densities out of literature range"
            z_score_assertion(neuron_dens, neuron_dens_litt - neuron_dens_tolerance, neuron_dens_litt + neuron_dens_tolerance, assertion_message)            
                 
            print(f"Finished {sc_factor}.", file=log_file)
            log_file.flush()

/gpfs/bbp.cscs.ch/ssd/slurmTmpFS/veraszto/2504452/ipykernel_62515/1828245804.py:78: UserWarning: total neuron densities out of literature range
  warnings.warn(assertion_message, UserWarning)


KeyboardInterrupt: 

GLIA VALIDATION

In [ ]:
%%time

from contextlib import redirect_stdout
log_file_path = f"{root_folder}/log/glia_print_messages.txt"
with open(log_file_path, "w") as log_file:
    with redirect_stdout(log_file):
        for sc_factor in [0.5, 0.6, 0.7, 0.8, 0.9, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6,1.7, 1.8, 1.9]: 
            try: del shuffled_combined_dataframes
            except NameError: pass
            
            file = os.path.join(f'{root_folder}scaled_densities_new_{sc_factor}.pickle')
            #Load region id volumes from volume_calc_from_template.ipynb
            with open(file, 'rb') as pickle_file:
                scaled_combined_result_dataframes = pickle.load(pickle_file)
        
            print("", file=log_file)
            print(f"Loaded pickle file for {sc_factor}.", file=log_file)
            log_file.flush()
            shuffled_combined_dataframes = create_combined_dataframe(scaled_combined_result_dataframes) #[celltypes]region
            print("Celltype/region dict is created.", file=log_file)
            log_file.flush()
            main()
            print("Total glia files for sc_factor is created.", file=log_file)
            glia_total, __ = nrrd.read(os.path.join(density_folder, "scaled_total_glia_densities.nrrd"))
            print("Total glia file loaded.", file=log_file)
            log_file.flush()

            # Assertion on total glia densities
            glia_dens = np.sum(glia_total) / whole_brain_annotation_dens # * voxel_volume
            glia_dens_diff = abs(glia_dens_litt - glia_dens)
            print("\nAssertion on total glia densities (/mm^3)")
            print_range_bar(glia_dens, glia_dens_litt - glia_dens_tolerance, glia_dens_litt + glia_dens_tolerance)
            assertion_message = "total glia densities out of literature range"
            z_score_assertion(glia_dens, glia_dens_litt - glia_dens_tolerance, glia_dens_litt + glia_dens_tolerance, assertion_message)            
        
            print(f"Finished {sc_factor}.", file=log_file)
            log_file.flush()


Process ForkPoolWorker-2154:
Process ForkPoolWorker-2155:
Process ForkPoolWorker-2142:
Process ForkPoolWorker-2152:
Process ForkPoolWorker-2147:
Process ForkPoolWorker-2146:
Process ForkPoolWorker-2151:
Process ForkPoolWorker-2144:
Process ForkPoolWorker-2145:
Process ForkPoolWorker-2143:
Process ForkPoolWorker-2159:
Process ForkPoolWorker-2150:
Process ForkPoolWorker-2157:
Process ForkPoolWorker-2139:
Process ForkPoolWorker-2160:
Process ForkPoolWorker-2116:
Process ForkPoolWorker-2141:
Process ForkPoolWorker-2098:
Process ForkPoolWorker-2120:
Process ForkPoolWorker-2148:
Process ForkPoolWorker-2149:
Process ForkPoolWorker-2156:
Process ForkPoolWorker-2153:
Process ForkPoolWorker-2158:
Process ForkPoolWorker-2138:
Process ForkPoolWorker-2090:
Process ForkPoolWorker-2089:
Process ForkPoolWorker-2115:
Process ForkPoolWorker-2091:
Process ForkPoolWorker-2105:
Process ForkPoolWorker-2088:
Process ForkPoolWorker-2117:
Process ForkPoolWorker-2122:
Process ForkPoolWorker-2083:
Process ForkPo

INHIBITORY VALIDATION

In [43]:
%%time
#0.5, 0.6, 0.7, 0.8, 0.9, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6,
from contextlib import redirect_stdout
log_file_path = f"{root_folder}/log/inh_print_messages.txt"
with open(log_file_path, "w") as log_file:
    with redirect_stdout(log_file):
        for sc_factor in [1.7, 1.8, 1.9]: 
            try: del shuffled_combined_dataframes
            except NameError: pass
            
            file = os.path.join(f'{root_folder}scaled_densities_new_{sc_factor}.pickle')
            #Load region id volumes from volume_calc_from_template.ipynb
            with open(file, 'rb') as pickle_file:
                scaled_combined_result_dataframes = pickle.load(pickle_file)
        
            print("", file=log_file)
            print(f"Loaded pickle file for {sc_factor}.", file=log_file)
            log_file.flush()
            shuffled_combined_dataframes = create_combined_dataframe(scaled_combined_result_dataframes) #[celltypes]region
            print("Celltype/region dict is created.", file=log_file)
            log_file.flush()
            main()
            print("Total INH files for sc_factor is created.", file=log_file)
            gad, __ = nrrd.read(os.path.join(density_folder, "scaled_total_inhibitory_densities.nrrd"))
            print("Total INH file loaded.", file=log_file)
            log_file.flush()
            # Assertion on total inhibitory neuron densities
            inhi_dens = np.sum(gad) / whole_brain_annotation_dens # * voxel_volume
            diff_inhi = abs(inhibitory_neuron_dens_litt - inhi_dens)
            print("\nAssertion on total inhibitory neuron densities (/mm^3)", file=log_file)
            print(r"/!\ Tolerance set to default for neuron density", file=log_file)
            # print("/!\ Default tolerance increased by a factor of 2.5")
            print_range_bar(inhi_dens, inhibitory_neuron_dens_litt - inhibitory_neuron_dens_tolerance, inhibitory_neuron_dens_litt + inhibitory_neuron_dens_tolerance)
            # assert (diff_inhi <= inhibitory_neuron_dens_tolerance * 2.5) #, f"diff_ini = " + str(diff_inhi) + "   |   tolerence = " + str(inhibitory_neuron_dens_tolerance)
            assertion_message = "total inhibitory neuron densities out of literature range"
            z_score_assertion(inhi_dens, inhibitory_neuron_dens_litt - inhibitory_neuron_dens_tolerance, inhibitory_neuron_dens_litt + inhibitory_neuron_dens_tolerance, assertion_message)
        
            print(f"Finished {sc_factor}.", file=log_file)
            log_file.flush()




CPU times: user 2min 45s, sys: 8.86 s, total: 2min 54s
Wall time: 30min 36s


In [ ]:
# Open a file in write mode
with open(os.path.join(density_folder, "output.txt"), "w") as f:
    # Save the current stdout so we can revert back later
    original_stdout = sys.stdout
    # Redirect stdout to the file
    sys.stdout = f

    for sc_factor in [1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0]: 
        try: del shuffled_combined_dataframes
        except NameError: pass
        
        file = os.path.join(f'{root_folder}scaled_densities_new_{sc_factor}.pickle')
        #Load region id volumes from volume_calc_from_template.ipynb
        with open(file, 'rb') as pickle_file:
            scaled_combined_result_dataframes = pickle.load(pickle_file)
    
        print()
        print("Loaded pickle file.", flush=True)
        shuffled_combined_dataframes = create_combined_dataframe(scaled_combined_result_dataframes) #[celltypes]region
    
        def main():
            # Define the parameters for each process
            tasks = [
                (neurontypes, "scaled_total_neuron_densities"),
                (nonneurontypes, "scaled_total_nonneuron_densities"),
                (exctypes, "scaled_total_excitatory_densities"),
                (inhtypes, "scaled_total_inhibitory_densities"),
                (astrotypes, "scaled_total_astrotypes_densities"),
                (microglia, "scaled_total_microglia_densities"),
                (oligos, "scaled_total_oligocyte_densities"),
                (glia, "scaled_total_glia_densities"),
                (exci_inhib_sum, "scaled_total_excinh_densities"),
                (celltypes, "scaled_total_celltypes_densities"),
            ]    
            # Create a multiprocessing Pool
            with mp.Pool(processes=mp.cpu_count()) as pool:
                results = pool.starmap(process_type, tasks)
            
            # Sequentially write the .nrrd files to avoid concurrent writes
            for result, file_name in results:
                print(file_name, flush=True)
                nrrd.write(f"{root_folder}{file_name}.nrrd", result)
    
        main()    
    
        cell, __ = nrrd.read(os.path.join(density_folder, "scaled_total_celltypes_densities.nrrd"))
        neuron, __ = nrrd.read(os.path.join(density_folder, "scaled_total_neuron_densities.nrrd"))
        
        exc, __ = nrrd.read(os.path.join(density_folder, "scaled_total_excitatory_densities.nrrd"))
        gad, __ = nrrd.read(os.path.join(density_folder, "scaled_total_inhibitory_densities.nrrd"))
        microglia, __ = nrrd.read(os.path.join(density_folder, "scaled_total_microglia_densities.nrrd"))
        nn, __ = nrrd.read(os.path.join(density_folder, "scaled_total_nonneuron_densities.nrrd"))
        olig, __ = nrrd.read(os.path.join(density_folder, "scaled_total_oligocyte_densities.nrrd"))
        astro, __ = nrrd.read(os.path.join(density_folder, "scaled_total_astrotypes_densities.nrrd"))
        glia, __ = nrrd.read(os.path.join(density_folder, "scaled_total_glia_densities.nrrd"))
        exci_inhib_sum, __ = nrrd.read(os.path.join(density_folder, "scaled_total_excinh_densities.nrrd"))
    
    
        z = 500
        plt.imshow(cell[z], cmap='coolwarm');
        # Define the coordinates
        x = 222
        y = 155
        
        name = region_map.get(annotation[z,y,x], "name", with_ascendants=False)
        
        plt.scatter(x, y, color='yellow', label='Point', s=10)
        # Add labels and title
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.title(f'The id at this point is {annotation[z,y,x]}, {name}, the value is {cell[z,y,x]}')
        # Show the plot
        plt.legend()
        plt.grid(True)
        plt.show()
        print(annotation[z,y,x], name, "value", cell[z,y,x],"(", z, y, x, ")", flush=True)
    
        # Assertion on total cell densities
        cell_dens = np.sum(cell) / whole_brain_annotation_dens# * voxel_volume
        cell_dens_diff = abs(cell_dens_litt - cell_dens)
        print("\nAssertion on total cell densities (/mm^3)", flush=True)
        print_range_bar(cell_dens, cell_dens_litt - cell_dens_tolerance, cell_dens_litt + cell_dens_tolerance)
        assertion_message = "total cell densities out of literature range"
        z_score_assertion(cell_dens, cell_dens_litt - cell_dens_tolerance, cell_dens_litt + cell_dens_tolerance, assertion_message)
    
        print()
    
        # Assertion on total neuron densities
        neuron_dens = np.sum(neuron) / whole_brain_annotation_dens # * voxel_volume
        neuron_dens_diff = abs(neuron_dens_litt - neuron_dens)
        print("\nAssertion on total neuron densities (/mm^3)", flush=True)
        print_range_bar(neuron_dens, neuron_dens_litt - neuron_dens_tolerance, neuron_dens_litt + neuron_dens_tolerance)
        assertion_message = "total neuron densities out of literature range"
        z_score_assertion(neuron_dens, neuron_dens_litt - neuron_dens_tolerance, neuron_dens_litt + neuron_dens_tolerance, assertion_message)
    
        print()
    
        # Assertion on total glia densities
        glia_dens = np.sum(glia) / whole_brain_annotation_dens # * voxel_volume
        glia_dens_diff = abs(glia_dens_litt - glia_dens)
        print("\nAssertion on total glia densities (/mm^3)", flush=True)
        print_range_bar(glia_dens, glia_dens_litt - glia_dens_tolerance, glia_dens_litt + glia_dens_tolerance)
        assertion_message = "total glia densities out of literature range"
        z_score_assertion(glia_dens, glia_dens_litt - glia_dens_tolerance, glia_dens_litt + glia_dens_tolerance, assertion_message)
    
        print()
    
        # Assertion on total inhibitory neuron densities
        inhi_dens = np.sum(gad) / whole_brain_annotation_dens # * voxel_volume
        diff_inhi = abs(inhibitory_neuron_dens_litt - inhi_dens)
        print("\nAssertion on total inhibitory neuron densities (/mm^3)", flush=True)
        print(r"/!\ Tolerance set to default for neuron density", flush=True)
        # print("/!\ Default tolerance increased by a factor of 2.5")
        print_range_bar(inhi_dens, inhibitory_neuron_dens_litt - inhibitory_neuron_dens_tolerance, inhibitory_neuron_dens_litt + inhibitory_neuron_dens_tolerance)
        # assert (diff_inhi <= inhibitory_neuron_dens_tolerance * 2.5) #, f"diff_ini = " + str(diff_inhi) + "   |   tolerence = " + str(inhibitory_neuron_dens_tolerance)
        assertion_message = "total inhibitory neuron densities out of literature range"
        z_score_assertion(inhi_dens, inhibitory_neuron_dens_litt - inhibitory_neuron_dens_tolerance, inhibitory_neuron_dens_litt + inhibitory_neuron_dens_tolerance, assertion_message)
    
        print()
    
        exci_inhib_sum_sum = np.sum(exci_inhib_sum) / whole_brain_annotation_dens # * voxel_volume
        diff_exci_inhib_sum = abs(neuron_dens_litt - exci_inhib_sum_sum)
        print("\nAssertion on sum of inhibitory + excitatory neuron densities (/mm^3)", flush=True)
        print_range_bar(exci_inhib_sum_sum, neuron_dens_litt - neuron_dens_tolerance, neuron_dens_litt + neuron_dens_tolerance)
        # print("/!\ Data not available")
        assertion_message = "sum of inhibitory + excitatory neuron densities out of literature range"
        z_score_assertion(exci_inhib_sum_sum, neuron_dens_litt - neuron_dens_tolerance, neuron_dens_litt + neuron_dens_tolerance, assertion_message)
    
        print("New Loop!", flush=True)
        
    # Reset stdout to its original state
    sys.stdout = original_stdout